# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
gmaps.configure(api_key= g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
data = pd.read_csv("weather_data.csv")
weather_data_df = pd.DataFrame(data)
weather_data_df.head()

,City,Cloudiness (%),Country,Date,Humidity (%),Latitude,Longitude,Max Temp (C°),Wind Speed (kmph)
0,Port Elizabeth,0,ZA,1588959251,93,-33.92,25.57,17.00,2.60
1,Olafsvik,0,IS,1588959251,66,64.89,-23.71,3.85,3.69
2,Hithadhoo,71,MV,1588959251,78,-0.60,73.08,29.65,6.65
3,East London,37,ZA,1588959228,81,-33.02,27.91,21.11,2.87
4,Dongsheng,100,CN,1588959252,43,39.82,109.98,10.91,5.35


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
locations  = weather_data_df[["Latitude","Longitude"]]

humidity = weather_data_df["Humidity (%)"]

In [4]:
fig = gmaps.figure(
    center = (15,0), 
    zoom_level=1.5
)

heat_layer = gmaps.heatmap_layer(
    locations,
    weights = humidity,
    dissipating = False,
    point_radius=2,
    max_intensity= float(weather_data_df["Humidity (%)"].max()),
)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
vacation_df = weather_data_df.loc[weather_data_df["Cloudiness (%)"] == 0]
vacation_df = vacation_df.loc[vacation_df["Wind Speed (kmph)"] < 16]
vacation_df = vacation_df.loc[vacation_df["Max Temp (C°)"] < 26]
vacation_df = vacation_df.loc[vacation_df["Max Temp (C°)"] > 21]
vacation_df = vacation_df.loc[vacation_df["Humidity (%)"] < 50]
vacation_df.dropna(inplace=True)
vacation_df["Location"] = vacation_df[['Latitude','Longitude']].apply(
    lambda x : '{},{}'.format(x[0],x[1]), axis=1)
vacation_df

,City,Cloudiness (%),Country,Date,Humidity (%),Latitude,Longitude,Max Temp (C°),Wind Speed (kmph),Location
174,Guerrero Negro,0,MX,1588959302,39,27.98,-114.06,25.64,2.00,"27.98,-114.06"
385,Saldanha,0,ZA,1588959363,44,-33.01,17.94,24.00,2.60,"-33.01,17.94"
389,Tura,0,IN,1588959364,40,25.52,90.22,23.88,1.73,"25.52,90.22"
455,Awjilah,0,LY,1588959386,18,29.11,21.29,25.28,4.87,"29.11,21.29"
550,Jardim,0,BR,1588959412,34,-21.48,-56.14,25.24,2.26,"-21.48,-56.14"


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
column_names = ["Hotel Name", "Lat", "Lng", "City"]

hotel_df = pd.DataFrame(columns = column_names)

In [7]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "rankby": "distance",
    "key": g_key,
    "keyword" : "hotel",
    "location" : "25.52,90.22",
}

response = requests.get(base_url, params=params).json()
response["results"][0] ["geometry"]["location"]["lat"]   


for index, row in vacation_df.iterrows():
    
    city_location = row["Location"]
    City = row["City"]
    Country = row["Country"]
    
    params["location"] = city_location
    
    print(f"Retrieving Results for hotels located in {City}.")
    response = requests.get(base_url, params=params).json()
    
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        
        hotel_df.loc[index, "Hotel Name"] = (results[0]['name'])
        hotel_df.loc[index, "City"] = City
        hotel_df.loc[index, "Hotel Name"] = (results[0]['name'])
        hotel_df.loc[index, "City"] = City
        hotel_df.loc[index, "Lat"] = (results[0]["geometry"]["location"]["lat"])
        hotel_df.loc[index, "Lng"] = (results[0]["geometry"]["location"]["lng"])
        hotel_df.loc[index, "Country"] = Country
        

    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    

Retrieving Results for hotels located in Guerrero Negro.
Closest hotel is Casa Laguna, Bed & Breakfast.
------------
Retrieving Results for hotels located in Saldanha.
Closest hotel is Hoedjiesbaai Hotel.
------------
Retrieving Results for hotels located in Tura.
Closest hotel is Hotel RIKMAN Continental.
------------
Retrieving Results for hotels located in Awjilah.
Closest hotel is Awjilah Tourist Hotel.
------------
Retrieving Results for hotels located in Jardim.
Closest hotel is Hotel Meridian.
------------


In [8]:
hotel_df.reset_index(drop=True)
hotel_locations = hotel_df[["Lat", "Lng"]]
hotel_df

,Hotel Name,Lat,Lng,City,Country
174,"Casa Laguna, Bed & Breakfast",27.9593,-114.055,Guerrero Negro,MX
385,Hoedjiesbaai Hotel,-33.0076,17.9459,Saldanha,ZA
389,Hotel RIKMAN Continental,25.5127,90.2176,Tura,IN
455,Awjilah Tourist Hotel,29.1682,21.2925,Awjilah,LY
550,Hotel Meridian,-21.4817,-56.1393,Jardim,BR


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotel_locations = hotel_df[["Lat", "Lng"]]
marker_layer = gmaps.marker_layer(hotel_locations, info_box_content=hotel_info)

In [10]:
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)


markers = gmaps.marker_layer(hotel_locations)

fig.add_layer(markers)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

In [11]:
# Add marker layer ontop of heat map
fig.add_layer(heat_layer)
fig.add_layer(markers)
fig.add_layer(marker_layer)
fig
# Display Map

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…